In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
#supply = pd.read_csv('20201007_da_co.processed.csv').set_index('id')
supply = pd.DataFrame([
    ['Bus 1', 100., 12.],
    ['Bus 2', 80., 20.],
], columns = ['node', 'capacity (MW)', 'offer ($/MW)'])
nsupply = len(supply)
print(supply.shape)
supply.head()

(2, 3)


,node,capacity (MW),offer ($/MW)
0,Bus 1,100.0,12.0
1,Bus 2,80.0,20.0


In [3]:
# demand = pd.read_csv('20201007_bids_cb.processed.csv').set_index('id')
demand = pd.DataFrame([
    ['Bus 2', 100., 40.],
    ['Bus 3', 50., 35.],
], columns = ['node', 'demand (MW)', 'bid ($/MW)'])
ndemand = len(demand)
print(demand.shape)
demand.head()

(2, 3)


,node,demand (MW),bid ($/MW)
0,Bus 2,100.0,40.0
1,Bus 3,50.0,35.0


In [4]:
nodes = sorted(set(supply['node']) | set(demand['node']))
nnodes = len(nodes)
print(nodes)

['Bus 1', 'Bus 2', 'Bus 3']


In [5]:
demand.groupby(demand['bid ($/MW)'] >= 2000.)['node'].count()

bid ($/MW)
False    2
Name: node, dtype: int64

In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 100., 500.] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
Bus 1-Bus 2,Bus 1,Bus 2,100.0,500.0
Bus 1-Bus 3,Bus 1,Bus 3,100.0,500.0
Bus 2-Bus 3,Bus 2,Bus 3,100.0,500.0


In [7]:
rev_lines = lines.copy()
rev_lines.source = lines.dest
rev_lines.dest = lines.source
bi_lines = pd.concat([lines, rev_lines]).set_index(['source', 'dest'], drop=False)
n_bi_lines = len(bi_lines)
bi_lines


source   dest  capacity (MW)  susceptance (S)
source dest                                               
Bus 1  Bus 2  Bus 1  Bus 2          100.0            500.0
       Bus 3  Bus 1  Bus 3          100.0            500.0
Bus 2  Bus 3  Bus 2  Bus 3          100.0            500.0
       Bus 1  Bus 2  Bus 1          100.0            500.0
Bus 3  Bus 1  Bus 3  Bus 1          100.0            500.0
       Bus 2  Bus 3  Bus 2          100.0            500.0

In [8]:
pd.concat([demand.groupby('node')[['demand (MW)']].sum(),
           supply.groupby('node')[['capacity (MW)']].sum()], axis='columns').fillna(0.)

,demand (MW),capacity (MW)
node,,
Bus 2,100.0,80.0
Bus 3,50.0,0.0
Bus 1,0.0,100.0


In [9]:
M = Model('kkt')

log = StringIO()
M.setLogHandler(log)

## variables

# primal variables
pD = M.variable('pD', ndemand, Domain.inRange(0., demand['demand (MW)'].values))
pG = M.variable('pG', nsupply, Domain.inRange(0., supply['capacity (MW)'].values))
theta = M.variable('theta', nnodes, Domain.inRange(
    [0.] + [-np.pi] * (nnodes - 1), 
    [0.] + [np.pi] * (nnodes - 1), 
))
# (theta reference value imposed as variable bound)

# line flow artificial variable, to simplify the MOSEK API calls
line_flows = M.variable('flows', n_bi_lines, Domain.unbounded())
src_idx = [nodes.index(x) for x in bi_lines.source]
dst_idx = [nodes.index(x) for x in bi_lines.dest]
phase_diff = Expr.sub(theta.pick(src_idx), theta.pick(dst_idx))
phase_flow = M.constraint(
    Expr.sub(line_flows, Expr.mulElm(bi_lines['susceptance (S)'].values, phase_diff)),
    Domain.equalsTo(0.)
)

#dual variables
muDup = M.variable('muDup', ndemand, Domain.greaterThan(0.))
muDdown = M.variable('muDdown', ndemand, Domain.greaterThan(0.))
muGup = M.variable('muGup', nsupply, Domain.greaterThan(0.))
muGdown = M.variable('muGdown', nsupply, Domain.greaterThan(0.))
etaup = M.variable('etaup', n_bi_lines, Domain.greaterThan(0.))
etadown = M.variable('etadown', n_bi_lines, Domain.greaterThan(0.))
lambdaN = M.variable('lambdaN', nnodes, Domain.unbounded())
gamma = M.variable('gamma', 1, Domain.unbounded())

## first order conditions

#dL/dpD
demand_idx = [nodes.index(x) for x in demand.node]
dldpd = Expr.sub(muDup, demand['bid ($/MW)'].values)
dldpd = Expr.sub(dldpd, muDdown)
dldpd = Expr.add(dldpd, lambdaN.pick(demand_idx))
pd_const = M.constraint('dldpd', dldpd, Domain.equalsTo(0.))

#dL/dpG
supply_idx = [nodes.index(x) for x in supply.node]
dldpg = Expr.add(muGup, supply['offer ($/MW)'].values)
dldpg = Expr.sub(dldpg, muGdown)
dldpg = Expr.sub(dldpg, lambdaN.pick(supply_idx))
pg_const = M.constraint('dldpg', dldpg, Domain.equalsTo(0.))

#dL/dtheta
theta_eqs = []
for n, n_node in enumerate(nodes):
    if n == 0:
        dldtheta = gamma
    else:
        dldtheta = Expr.zeros(1)
    for m_node in bi_lines.xs(n_node).index:
        m = nodes.index(m_node)

        n_m_idx = bi_lines.index.get_loc((n_node, m_node))
        m_n_idx = bi_lines.index.get_loc((m_node, n_node))
        nm_duals = Expr.sub(etaup.index(n_m_idx), 
                            etaup.index(m_n_idx))
        nm_duals = Expr.sub(nm_duals, etadown.index(n_m_idx))
        nm_duals = Expr.add(nm_duals, etadown.index(m_n_idx))
        nm_duals = Expr.add(nm_duals, lambdaN.index(n))
        nm_duals = Expr.sub(nm_duals, lambdaN.index(m))
        dldtheta = Expr.add(
            dldtheta, 
            Expr.mul(bi_lines.loc[(n_node, m_node), 'susceptance (S)'],
                     nm_duals))
    theta_eqs.append(
        M.constraint('dldtheta%d' % n, dldtheta, Domain.equalsTo(0.)))

## primal equality constraints

# node balance equations    
node_balance_eqs = []
for node in nodes:
    supply_idx = np.flatnonzero(supply.node == node).astype('int32')
    node_supply = Expr.sum(pG.pick(supply_idx))
    demand_idx = np.flatnonzero(demand.node == node).astype('int32')
    node_demand = Expr.sum(pD.pick(demand_idx))
    balance = Expr.sub(node_demand, node_supply)
    out_line_idx = np.flatnonzero(bi_lines.source == node).astype('int32')
    balance = Expr.add(balance, Expr.sum(line_flows.pick(out_line_idx)))
    in_line_idx = np.flatnonzero(bi_lines.dest == node).astype('int32')
    balance = Expr.sub(balance, Expr.sum(line_flows.pick(in_line_idx)))
    balance_eq = M.constraint(node + 'balance', balance, Domain.equalsTo(0.))
    node_balance_eqs.append(balance_eq)
    
## bilinearity constraints
bigM = 5e5  # It's M. It's Big. It's Big-M!
binary_domain = Domain.binary()
# binary_domain = Domain.inRange(0., 1.)

# 0 ≤ −pd + PD ⊥ μdup ≥ 0 ∀d
z_upper_d = M.variable('z_upper_d', ndemand, binary_domain)
M.constraint(Expr.sub(Expr.sub(demand['demand (MW)'].values, pD),
                      Expr.mul(bigM, z_upper_d)), Domain.lessThan(0.))
not_z_upper_d = Expr.sub(1., z_upper_d)
M.constraint(Expr.sub(muDup,
                      Expr.mul(bigM, not_z_upper_d)), Domain.lessThan(0.))

# 0 ≤ pd ⊥ μddown ≥ 0 ∀d
z_lower_d = M.variable('z_lower_d', ndemand, binary_domain)
M.constraint(Expr.sub(pD,
                      Expr.mul(bigM, z_lower_d)), Domain.lessThan(0.))
not_z_lower_d = Expr.sub(1., z_lower_d)
M.constraint(Expr.sub(muDdown,
                      Expr.mul(bigM, not_z_lower_d)), Domain.lessThan(0.))

# 0 ≤ −pg + PG ⊥ μgup ≥ 0 ∀g
z_upper_g = M.variable('z_upper_g', nsupply, binary_domain)
M.constraint(Expr.sub(Expr.sub(supply['capacity (MW)'].values, pG),
                      Expr.mul(bigM, z_upper_g)), Domain.lessThan(0.))
not_z_upper_g = Expr.sub(1., z_upper_g)
M.constraint(Expr.sub(muGup,
                      Expr.mul(bigM, not_z_upper_g)), Domain.lessThan(0.))

# 0 ≤ pg ⊥ μgdown ≥ 0 ∀g
z_lower_g = M.variable('z_lower_g', nsupply, binary_domain)
M.constraint(Expr.sub(pG,
                      Expr.mul(bigM, z_lower_g)), Domain.lessThan(0.))
not_z_lower_g = Expr.sub(1., z_lower_g)
M.constraint(Expr.sub(muGdown,
                      Expr.mul(bigM, not_z_lower_g)), Domain.lessThan(0.))

# 0 ≤ −Bn,m (θn − θm ) + Fn,m ⊥ ηupn,m ≥ 0
z_upper_eta = M.variable('z_upper_eta', n_bi_lines, binary_domain)
M.constraint(Expr.sub(Expr.sub(bi_lines['capacity (MW)'].values, line_flows),
                      Expr.mul(bigM, z_upper_eta)), Domain.lessThan(0.))
not_z_upper_eta = Expr.sub(1., z_upper_eta)
M.constraint(Expr.sub(etaup,
                      Expr.mul(bigM, not_z_upper_eta)), Domain.lessThan(0.))

# 0 ≤ Bn,m (θn − θm ) ⊥ ηlown,m ≥ 0
z_lower_eta = M.variable('z_lower_eta', n_bi_lines, binary_domain)
M.constraint(Expr.sub(line_flows,
                      Expr.mul(bigM, z_lower_eta)), Domain.lessThan(0.))
not_z_lower_eta = Expr.sub(1., z_lower_eta)
M.constraint(Expr.sub(etadown,
                      Expr.mul(bigM, not_z_lower_eta)), Domain.lessThan(0.))

## dummy objective constraint
M.objective('obj', ObjectiveSense.Maximize, M.variable(Domain.equalsTo(0.)))
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal:
    M.writeTask('proj1kkt.opf')

demand['consumed (MW)'] = pD.level()
supply['supplied (MW)'] = pG.level()
bi_lines['flow (MW)'] = line_flows.level()
buses = pd.DataFrame({
    'volt. angle (rad)': theta.level(),
    'node price ($/MW)': lambdaN.level()
}, index=nodes)

In [10]:
summary = pd.concat([
    buses['node price ($/MW)'],
    demand.groupby('node')['consumed (MW)'].sum(),
    supply.groupby('node')['supplied (MW)'].sum(),
], axis=1).fillna(0.)
summary.columns = ['price ($/MW)', 'consumed (MW)', 'supplied (MW)']
display(summary)
print(summary.to_latex())

,price ($/MW),consumed (MW),supplied (MW)
Bus 1,12.0,0.0,70.0
Bus 2,20.0,100.0,80.0
Bus 3,35.0,50.0,0.0


\begin{tabular}{lrrr}
\toprule
{} &  price (\$/MW) &  consumed (MW) &  supplied (MW) \\
\midrule
Bus 1 &          12.0 &            0.0 &           70.0 \\
Bus 2 &          20.0 &          100.0 &           80.0 \\
Bus 3 &          35.0 &           50.0 &            0.0 \\
\bottomrule
\end{tabular}



In [11]:
print('Energy scheduled: {:g}'.format(demand['consumed (MW)'].sum()))
print('Objective value: {:g}'.format(M.primalObjValue()))
display(bi_lines)
display(buses)
display(demand.sort_values('bid ($/MW)'))
display(supply.sort_values('offer ($/MW)'))

Energy scheduled: 150
Objective value: 0


source   dest  capacity (MW)  susceptance (S)  flow (MW)
source dest                                                          
Bus 1  Bus 2  Bus 1  Bus 2          100.0            500.0       15.0
       Bus 3  Bus 1  Bus 3          100.0            500.0       20.0
Bus 2  Bus 3  Bus 2  Bus 3          100.0            500.0        5.0
       Bus 1  Bus 2  Bus 1          100.0            500.0      -15.0
Bus 3  Bus 1  Bus 3  Bus 1          100.0            500.0      -20.0
       Bus 2  Bus 3  Bus 2          100.0            500.0       -5.0

,volt. angle (rad),node price ($/MW)
Bus 1,0.00,12.0
Bus 2,-0.03,20.0
Bus 3,-0.04,35.0


,node,demand (MW),bid ($/MW),consumed (MW)
1,Bus 3,50.0,35.0,50.0
0,Bus 2,100.0,40.0,100.0


,node,capacity (MW),offer ($/MW),supplied (MW)
0,Bus 1,100.0,12.0,70.0
1,Bus 2,80.0,20.0,80.0


In [12]:
print(log.getvalue())

Problem
  Name                   : kkt             
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 56              
  Cones                  : 0               
  Scalar variables       : 59              
  Matrix variables       : 0               
  Integer variables      : 20              

Optimizer started.
Mixed integer optimizer started.
Threads used: 6
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 39 variables, 39 constraints, 100 non-zeros
Presolved problem: 0 general integer, 14 binary, 25 continuous
Clique table size: 8
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   -0.0000000000e+00    NA          0.0   
Cut generation started.
0        1        1        0        NA                   -0.0000000000e+00    NA          0.0   
Cut generation terminated. Time = 0.00
9  